In [1]:
import zipfile
import os


with zipfile.ZipFile("/content/GPUPy.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [2]:
with zipfile.ZipFile("/content/GPUPy.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [6]:
import time
import numpy as np
import cupy as cp
from GPUPy.src.numerical_methods.optimization import minimize_wrapper, minimize_scalar_wrapper

def setup_scalar_problem():
    def rosenbrock_scalar(x):
        return (1 - x)**2 + 100*(0 - x**2)**2

    return rosenbrock_scalar, -2.0  # Function and initial guess

def setup_vector_problem(dim=100):
    def rosenbrock(x):
        return np.sum(100.0*(x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

    x0 = np.zeros(dim)
    x0[::2] = 1.5
    x0[1::2] = -0.5
    return rosenbrock, x0

def run_benchmark():
    print("=== Optimization Benchmark (CPU vs GPU) ===")
    print("Note: GPU acceleration is only for function evaluation")
    print("The optimization algorithm itself runs on CPU\n")

    # Benchmark scalar optimization
    print("\n1. Scalar Optimization (Rosenbrock function)")
    func, x0 = setup_scalar_problem()

    # CPU
    start = time.time()
    res_cpu = minimize_scalar_wrapper(func, use_gpu=False, method='brent')
    cpu_time = time.time() - start

    # GPU (if available)
    if cp is not None:
        start = time.time()
        res_gpu = minimize_scalar_wrapper(func, use_gpu=True, method='brent')
        gpu_time = time.time() - start

        print(f"CPU Time: {cpu_time:.4f}s, Solution: {res_cpu.x:.6f}")
        print(f"GPU Time: {gpu_time:.4f}s, Solution: {res_gpu.x:.6f}")
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    else:
        print(f"CPU Time: {cpu_time:.4f}s, Solution: {res_cpu.x:.6f}")
        print("CuPy not available - skipping GPU test")

    # Benchmark multivariate optimization
    dim = 100  # Problem dimension
    print(f"\n2. Multivariate Optimization ({dim}D Rosenbrock function)")
    func, x0 = setup_vector_problem(dim)

    # CPU
    start = time.time()
    res_cpu = minimize_wrapper(func, x0, use_gpu=False, method='L-BFGS-B')
    cpu_time = time.time() - start

    # GPU (if available)
    if cp is not None:
        start = time.time()
        res_gpu = minimize_wrapper(func, x0, use_gpu=True, method='L-BFGS-B')
        gpu_time = time.time() - start

        print(f"CPU Time: {cpu_time:.4f}s, Final fval: {res_cpu.fun:.6f}")
        print(f"GPU Time: {gpu_time:.4f}s, Final fval: {res_gpu.fun:.6f}")
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")

        # Verify solutions are similar
        if not np.allclose(res_cpu.x, res_gpu.x, rtol=1e-2):
          print("WARNING: CPU and GPU solutions differ significantly!")
    else:
        print(f"CPU Time: {cpu_time:.4f}s, Final fval: {res_cpu.fun:.6f}")
        print("CuPy not available - skipping GPU test")

if __name__ == "__main__":
    run_benchmark()






=== Optimization Benchmark (CPU vs GPU) ===
Note: GPU acceleration is only for function evaluation
The optimization algorithm itself runs on CPU


1. Scalar Optimization (Rosenbrock function)
CPU Time: 0.0005s, Solution: 0.161262
GPU Time: 0.0067s, Solution: 0.161262
Speedup: 0.07x

2. Multivariate Optimization (100D Rosenbrock function)
CPU Time: 1.3612s, Final fval: 70.638377
GPU Time: 8.6634s, Final fval: 71.442128
Speedup: 0.16x
